In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('bank-full.csv',delimiter=";", na_values="unknown")
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,NaN,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,NaN,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,NaN,no


# Looking at variables that are skewed

In [ ]:
df.skew()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


age          0.684818
balance      8.360308
day          0.093079
duration     3.144318
campaign     4.898650
pdays        2.615715
previous    41.846454
dtype: float64

balance, duration, campaign, pdays and previous can be transfomred using various methods if the reason for the skewed data is not outliers 

first try log transformation

In [ ]:
columns_skewed = ['balance', 'duration', 'campaign', 'pdays', 'previous']
new_df = df.copy()
new_df = new_df[columns_skewed].apply(lambda x: np.log(x))
new_df.skew()

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


balance          NaN
duration         NaN
campaign    0.880211
pdays      -2.606166
previous         NaN
dtype: float64

log transformation works well with campaign but nothing else, works poorly in columns with a value of 0 as the transformation makes the value equal to infinity, which skews the data very highly resulting in NaN values

---



Try reduce skewness of the data using squared root method

In [ ]:
df2 = pd.DataFrame().assign(balance = df['balance'], duration = df['duration'], campaign = df['campaign'], pdays = df['pdays'], previous = df['previous'])

scaled_df = np.sqrt(df2)
scaled_df.skew()



balance     1.901328
duration    1.173744
campaign    2.206992
pdays      -0.292640
previous    2.880045
dtype: float64

reduce skewness on all, but by too little. Also pdays contains negative values so negative values are changed to Nan

Try reduce skewness using cube root method 

In [ ]:
scaled2_df = np.cbrt(df2)
scaled2_df.skew()



balance    -0.078187
duration    0.640146
campaign    1.653048
pdays       1.761872
previous    2.056920
dtype: float64

This method works much better to reduce skewness, however for campaign a log transformation is better

The last method is the box cox transformation

In [ ]:
from scipy import stats

scaled3_df = stats.boxcox(df2['campaign'])[0]
print(pd.Series(scaled3_df).skew())


0.16630337139412119


Method works very well, however only works on campaign as all data is positive

Overall it seems all data skewness can be reduced using the cube root method howver for campaign the boxcox method can be used to improve greater. Also hopefully removing some outliers will lead to a greater reduction in skewness as well